In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandasql as psql # for SQL

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

database_path = "/kaggle/input/videogamesales/vgsales.csv"

/kaggle/input/videogamesales/vgsales.csv


In [4]:
# Python
data = pd.read_csv("/kaggle/input/videogamesales/vgsales.csv")
data.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [5]:
data.info()

data['Platform'].value_counts() #Plataform

data.Genre.unique() #Genre

data['Publisher'].value_counts() #Publisher

data['Global_Sales'].describe() #Global_Sales

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


count    16598.000000
mean         0.537441
std          1.555028
min          0.010000
25%          0.060000
50%          0.170000
75%          0.470000
max         82.740000
Name: Global_Sales, dtype: float64

# How many PS4 sales are in each region?

In [6]:
query1 = psql.sqldf("""
                    SELECT SUM(JP_Sales) AS Num_JPN_Sales,
                    SUM(EU_Sales) AS Num_EU_Sales,
                    SUM(NA_Sales) AS Num_NA_Sales FROM data
                    WHERE Platform = 'PS4' 
                    
                    """)
query1.head()

,Num_JPN_Sales,Num_EU_Sales,Num_NA_Sales
0,14.3,123.7,96.8


# Which are the three best games sold in North America?

In [7]:
query2 = psql.sqldf("""
                    SELECT Name AS Game, NA_Sales  FROM data
                    GROUP BY Name
                    ORDER BY NA_Sales DESC
                    LIMIT 3
                    
                    """)
query2.head()

,Game,NA_Sales
0,Wii Sports,41.49
1,Super Mario Bros.,29.08
2,Duck Hunt,26.93


# Which are the year, the game and the genre with the most sales?

In [8]:
query3 = psql.sqldf("""
                    SELECT Global_Sales, Year, Genre, Name  FROM data
                    GROUP BY Genre
                    ORDER BY Global_sales DESC
                    LIMIT 5
                    
                    """)
query3.head()

,Global_Sales,Year,Genre,Name
0,82.74,2006.0,Sports,Wii Sports
1,40.24,1985.0,Platform,Super Mario Bros.
2,35.82,2008.0,Racing,Mario Kart Wii
3,31.37,1996.0,Role-Playing,Pokemon Red/Pokemon Blue
4,30.26,1989.0,Puzzle,Tetris


# We want to study just the Adventure games in the year 2015

In [9]:
query4 = psql.sqldf("""
                    SELECT * FROM data
                    WHERE Genre = 'Adventure' AND Year = 2015
                    GROUP BY Name
                    
                    """)
query4.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,14435,Alia's Carnival! Sacrament,PSV,2015.0,Adventure,dramatic create,0.00,0.00,0.03,0.00,0.03
1,15560,Amatsumi Sora ni! Kumo no Hatate ni,PSV,2015.0,Adventure,Prototype,0.00,0.00,0.02,0.00,0.02
2,8234,Back to the Future: The Game,PS4,2015.0,Adventure,Telltale Games,0.10,0.04,0.00,0.03,0.17
3,12282,Broken Sword 5: The Serpent's Curse,PS4,2015.0,Adventure,Revolution Software,0.00,0.06,0.00,0.01,0.06
4,9513,Brothers: A Tale of Two Sons,PS4,2015.0,Adventure,505 Games,0.03,0.08,0.00,0.02,0.13


# Display the number of shooter games with more than 1.3 sales in Europe

In [10]:
query5 = psql.sqldf("""
                    SELECT COUNT(DISTINCT(Name)) AS num_Games FROM data
                    WHERE Genre = 'Shooter' AND EU_Sales > 1.3
                    ORDER BY EU_Sales DESC
                    
                    """)
query5.head()

,num_Games
0,32
